In [1]:
import pandas as pd # library for data analsysis
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy #uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering libraryt requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe



print('Libraries imported.')

Libraries imported.


In [2]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Neighbourhood,Avg Price per sqft
0,0,AP SE Board Hyderabad,4445.0
1,1,Ac Guards Hyderabad,6492.0
2,2,Ahmed Nagar Hyderabad,2343.0
3,3,Amberpet Hyderabad,2222.0
4,4,Ameerpet Hyderabad,9628.0


In [3]:
hyderabad_neighbourhoods=df_data_3.dropna(axis=0,how='any')
hyderabad_neighbourhoods.head()


,Unnamed: 0,Neighbourhood,Avg Price per sqft
0,0,AP SE Board Hyderabad,4445.0
1,1,Ac Guards Hyderabad,6492.0
2,2,Ahmed Nagar Hyderabad,2343.0
3,3,Amberpet Hyderabad,2222.0
4,4,Ameerpet Hyderabad,9628.0


In [4]:
hyderabad_neighbourhoods=hyderabad_neighbourhoods[hyderabad_neighbourhoods.Neighbourhood != 'Chaitanyapuri Colony Hyderabad']
hyderabad_neighbourhoods.drop(['Unnamed: 0'],axis=1,inplace = True)


### Locating Google Office on Map ###

In [5]:
office_address = 'Kondapur Village, Hyderabad'

geolocator = Nominatim(user_agent="hyd_explorer")
location = geolocator.geocode(office_address)
office_latitude = location.latitude
office_longitude = location.longitude
print('The geograpical coordinate of Google office are {}, {}.'.format(office_latitude, office_longitude))


The geograpical coordinate of Google office are 17.4587912, 78.3730556.


In [6]:
# create map of Hyderabad showing Google office using latitude and longitude values
map_Hyderabad = folium.Map(location=[office_latitude, office_longitude], zoom_start=12)

# add markers to map


map_Hyderabad

### Now we shall get the latitude and longitudes of all the neighbourhoods and locate them on the map ###



In [7]:
latitude=[]
longitude =[]

for i in range(len(hyderabad_neighbourhoods)):
    geolocator = Nominatim(user_agent="latlan_explorer")
    location = geolocator.geocode(hyderabad_neighbourhoods.iloc[i,0])
    latitude.append(location.latitude)
    longitude.append(location.longitude)

hyderabad_neighbourhoods['latitude'] = latitude
hyderabad_neighbourhoods['longitude'] = longitude
hyderabad_neighbourhoods.head()

,Neighbourhood,Avg Price per sqft,latitude,longitude
0,AP SE Board Hyderabad,4445.0,17.493930,78.402257
1,Ac Guards Hyderabad,6492.0,17.399877,78.458051
2,Ahmed Nagar Hyderabad,2343.0,17.404066,78.444229
3,Amberpet Hyderabad,2222.0,17.390263,78.516481
4,Ameerpet Hyderabad,9628.0,17.437501,78.448251


In [8]:
# create map of Hyderabad showing Google office using latitude and longitude values
map_Hyderabad = folium.Map(location=[office_latitude, office_longitude], zoom_start=12)

# add markers to map
folium.Marker(
    [office_latitude, office_longitude],
    radius=10,
    icon=folium.Icon(color='red'),
    popup='Google'
).add_to(map_Hyderabad)

for lat, lng, neighborhood in zip(hyderabad_neighbourhoods['latitude'] ,hyderabad_neighbourhoods['longitude'] ,hyderabad_neighbourhoods['Neighbourhood']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=neighborhood,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_Hyderabad)
map_Hyderabad

### Calculate the distance from each neighbourhood to office ###

In [9]:
# Importing the geodesic module from the library 
from geopy.distance import geodesic 

distance_from_office = []

for i in range(len(hyderabad_neighbourhoods)):
    distance_from_office.append(geodesic((office_latitude,office_longitude),(hyderabad_neighbourhoods.iloc[i,2],hyderabad_neighbourhoods.iloc[i,3])).km)

hyderabad_neighbourhoods['Distance in kms'] = distance_from_office
hyderabad_neighbourhoods.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True) 
hyderabad_neighbourhoods["Distance in kms"]=hyderabad_neighbourhoods["Distance in kms"].astype(int)
hyderabad_neighbourhoods.head()

,Neighborhood,Avg Price per sqft,latitude,longitude,Distance in kms
0,AP SE Board Hyderabad,4445.0,17.493930,78.402257,4
1,Ac Guards Hyderabad,6492.0,17.399877,78.458051,11
2,Ahmed Nagar Hyderabad,2343.0,17.404066,78.444229,9
3,Amberpet Hyderabad,2222.0,17.390263,78.516481,17
4,Ameerpet Hyderabad,9628.0,17.437501,78.448251,8


### Now let us visualise how distance and price vary for each neighbourhood ###

In [10]:
CLIENT_ID = '5ORTUHZURQD5QMP12DBAQJB1F00Y03VY2N04N1GJDPFLNQK5' # your Foursquare ID
CLIENT_SECRET = 'KMHDM5JOWMYCQXIMVKXRZ322YOTTCGTH1MEF1K0IESO34WPX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5ORTUHZURQD5QMP12DBAQJB1F00Y03VY2N04N1GJDPFLNQK5
CLIENT_SECRET:KMHDM5JOWMYCQXIMVKXRZ322YOTTCGTH1MEF1K0IESO34WPX


In [11]:
hyderabad_neighbourhoods.loc[0, 'Neighborhood']

'AP SE Board Hyderabad'

In [12]:
neighborhood_latitude = hyderabad_neighbourhoods.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = hyderabad_neighbourhoods.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = hyderabad_neighbourhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of AP SE Board Hyderabad are 17.4939302, 78.4022566.


In [13]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=5ORTUHZURQD5QMP12DBAQJB1F00Y03VY2N04N1GJDPFLNQK5&client_secret=KMHDM5JOWMYCQXIMVKXRZ322YOTTCGTH1MEF1K0IESO34WPX&v=20180605&ll=17.4939302,78.4022566&radius=500&limit=100'

In [14]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb0895cc546f3001c7f534a'},
 'response': {'headerLocation': 'Hydernagar',
  'headerFullLocation': 'Hydernagar, Hyderabad',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 21,
  'suggestedBounds': {'ne': {'lat': 17.498430204500007,
    'lng': 78.40696601873721},
   'sw': {'lat': 17.489430195499995, 'lng': 78.3975471812628}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51777e62498e1f75b5362d6f',
       'name': 'Paradise Restaurant',
       'location': {'address': 'Kukatpally Housing Board',
        'crossStreet': 'Towards Ushamullapudi Hospital Road.',
        'lat': 17.49336431759921,
        'lng': 78.40415464796615,
        'labeledLatLngs': [{'label': 'display',
          'lat': 17.49336431759921,
          'lng

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Paradise Restaurant,Indian Restaurant,17.493364,78.404155
1,K S Bakery,Bakery,17.494709,78.401117
2,Chaitanya Fine Dining,Food,17.493090,78.402392
3,Cream Stone Concepts,Ice Cream Shop,17.492800,78.404371
4,One Place Hotel,Hotel Bar,17.495338,78.400107


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

21 venues were returned by Foursquare.


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)          

In [19]:
hyderabad_venues = getNearbyVenues(names=hyderabad_neighbourhoods['Neighborhood'],
                                       latitudes=hyderabad_neighbourhoods['latitude'],
                                       longitudes=hyderabad_neighbourhoods['longitude']
                                      )

AP SE Board Hyderabad
Ac Guards Hyderabad
Ahmed Nagar Hyderabad
Amberpet Hyderabad
Ameerpet Hyderabad
Anand Nagar Hyderabad
Ashok Nagar Hyderabad
Asif Nagar Hyderabad
BK Guda Hyderabad
Banjara Hills Hyderabad
Barkatpura Hyderabad
Basheer Bagh Hyderabad
Bazar Ghat Hyderabad
Begum Bazar Hyderabad
Begumpet Hyderabad
Bhavani Nagar Hyderabad
Boiguda Hyderabad
Bholakpur Hyderabad
Bolarum Hyderabad
Bowenpally Hyderabad
Chanchalguda Hyderabad
Chandanagar Hyderabad
Chikkadapally Market Hyderabad
Chilkalguda Hyderabad
Dabirpura Hyderabad
Dattatreya Colony Hyderabad
Dilsukhnagar colony Hyderabad
East Maredpally Hyderabad
Erragadda Hyderabad
Punjagutta Hyderabad
Fatehnagar Hyderabad
Gachibowli Hyderabad
Golconda Hyderabad
Himayatnagar Hyderabad
Housing Board Colony Hyderabad
Humayun Nagar Hyderabad
JNTU Hyderabad
Jubilee Hills Hyderabad
Kavadiguda Hyderabad
Kompally Hyderabad
Kondapur Hyderabad
Kukatpally Hyderabad
Khairatabad Hyderabad
Lalitha nagar Colony Hyderabad
Lalaguda Hyderabad
Malakpet Co

In [20]:
print(hyderabad_venues.shape)
hyderabad_venues.head()

(636, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AP SE Board Hyderabad,17.49393,78.402257,Paradise Restaurant,17.493364,78.404155,Indian Restaurant
1,AP SE Board Hyderabad,17.49393,78.402257,K S Bakery,17.494709,78.401117,Bakery
2,AP SE Board Hyderabad,17.49393,78.402257,Chaitanya Fine Dining,17.493090,78.402392,Food
3,AP SE Board Hyderabad,17.49393,78.402257,Cream Stone Concepts,17.492800,78.404371,Ice Cream Shop
4,AP SE Board Hyderabad,17.49393,78.402257,One Place Hotel,17.495338,78.400107,Hotel Bar


In [21]:
hyderabad_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
AP SE Board Hyderabad,21,21,21,21,21,21
Ac Guards Hyderabad,4,4,4,4,4,4
Ahmed Nagar Hyderabad,7,7,7,7,7,7
Amberpet Hyderabad,3,3,3,3,3,3
Ameerpet Hyderabad,11,11,11,11,11,11
Ashok Nagar Hyderabad,4,4,4,4,4,4
BK Guda Hyderabad,4,4,4,4,4,4
Balanagar Hyderabad,4,4,4,4,4,4
Banjara Hills Hyderabad,38,38,38,38,38,38


In [22]:
# one hot encoding
hyderabad_onehot = pd.get_dummies(hyderabad_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hyderabad_onehot['Neighborhood'] =hyderabad_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hyderabad_onehot.columns[-1]] + list(hyderabad_onehot.columns[:-1])
hyderabad_onehot = hyderabad_onehot[fixed_columns]

hyderabad_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport Food Court,American Restaurant,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Bus Stop,Butcher,Cable Car,Café,Candy Store,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Currency Exchange,Deli / Bodega,Department Store,Dessert Shop,Diner,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hookah Bar,Hostel,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Men's Store,Metro Station,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Outdoors & Recreation,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Pub,Restaurant,Sandwich Place,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Supermarket,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,AP SE Board Hyderabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,AP SE Board Hyderabad,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,AP SE Board Hyderabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,AP SE Board Hyderabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,AP SE Board Hyderabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
hyderabad_grouped = hyderabad_onehot.groupby('Neighborhood').mean().reset_index()
hyderabad_grouped_grouped = hyderabad_onehot.groupby('Neighborhood').mean().reset_index()
hyderabad_grouped

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport Food Court,American Restaurant,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Bus Stop,Butcher,Cable Car,Café,Candy Store,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Currency Exchange,Deli / Bodega,Department Store,Dessert Shop,Diner,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hookah Bar,Hostel,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Men's Store,Metro Station,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Outdoors & Recreation,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Pub,Restaurant,Sandwich Place,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Supermarket,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,AP SE Board Hyderabad,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.095238,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.00,0.000000,0.000000,0.047619,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.095238,0.00,0.00,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.047619,0.190476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.047619,0.047619,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,Ac Guards Hyderabad,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,Ahmed Nagar Hyderabad,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.285714,0

In [24]:
num_top_venues = 5

for hood in hyderabad_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = hyderabad_grouped[hyderabad_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hyderabad_grouped['Neighborhood']

for ind in np.arange(hyderabad_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hyderabad_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AP SE Board Hyderabad,Indian Restaurant,Fast Food Restaurant,Bakery,Shop & Service,Clothing Store,Bus Station,Pizza Place,Breakfast Spot,Sandwich Place,Middle Eastern Restaurant
1,Ac Guards Hyderabad,Hyderabadi Restaurant,Hotel,Bakery,Women's Store,Deli / Bodega,Electronics Store,Dumpling Restaurant,Diner,Dessert Shop,Department Store
2,Ahmed Nagar Hyderabad,Snack Place,Café,Indian Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Women's Store,Dumpling Restaurant,Diner,Dessert Shop,Department Store
3,Amberpet Hyderabad,Movie Theater,IT Services,Indian Restaurant,Women's Store,Farmers Market,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Currency Exchange
4,Ameerpet Hyderabad,Indian Restaurant,Supermarket,Hotel,Diner,Buffet,Food Court,Sandwich Place,Restaurant,Vegetarian / Vegan Restaurant,Cupcake Shop
5,Ashok Nagar Hyderabad,Park,Restaurant,Dessert Shop,Brewery,Women's Store,Cocktail Bar,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop
6,BK Guda Hyderabad,Indian Restaurant,Bakery,Snack Place,Women's Store,Dumpling Restaurant,Diner,Dessert Shop,Department Store,Deli / Bodega,Farmers Market
7,Balanagar Hyderabad,Cocktail Bar,Metro Station,Market,Fish Market,Electronics Store,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop
8,Banjara Hills Hyderabad,Coffee Shop,Furniture / Home Store,Indian Restaurant,Bakery,Sandwich Place,Shoe Store,Boutique,Ice Cream Shop,Deli / Bodega,Bar
9,Barkatpura Hyderabad,Coffee Shop,Gym,Women's Store,Hotel,Platform,Bank,Café,Motel,Train Station,Cosmetics Shop


In [27]:
neighborhoods_venues_sorted.shape

(80, 11)

In [28]:
df_left = pd.merge(neighborhoods_venues_sorted,hyderabad_neighbourhoods, on='Neighborhood', how='left')
df_cluster = df_left
df_left


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Avg Price per sqft,latitude,longitude,Distance in kms
0,AP SE Board Hyderabad,Indian Restaurant,Fast Food Restaurant,Bakery,Shop & Service,Clothing Store,Bus Station,Pizza Place,Breakfast Spot,Sandwich Place,Middle Eastern Restaurant,4445.0,17.493930,78.402257,4
1,Ac Guards Hyderabad,Hyderabadi Restaurant,Hotel,Bakery,Women's Store,Deli / Bodega,Electronics Store,Dumpling Restaurant,Diner,Dessert Shop,Department Store,6492.0,17.399877,78.458051,11
2,Ahmed Nagar Hyderabad,Snack Place,Café,Indian Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Women's Store,Dumpling Restaurant,Diner,Dessert Shop,Department Store,2343.0,17.404066,78.444229,9
3,Amberpet Hyderabad,Movie Theater,IT Services,Indian Restaurant,Women's Store,Farmers Market,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Currency Exchange,2222.0,17.390263,78.516481,17
4,Ameerpet Hyderabad,Indian Restaurant,Supermarket,Hotel,Diner,Buffet,Food Court,Sandwich Place,Restaurant,Vegetarian / Vegan Restaurant,Cupcake Shop,9628.0,17.437501,78.448251,8
5,Ashok Nagar Hyderabad,Park,Restaurant,Dessert Shop,Brewery,Women's Store,Cocktail Bar,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,6106.0,17.405826,78.490260,13
6,BK Guda Hyderabad,Indian Restaurant,Bakery,Snack Place,Women's Store,Dumpling Restaurant,Diner,Dessert Shop,Department Store,Deli / Bodega,Farmers Market,3034.0,17.446628,78.446289,7
7,Balanagar Hyderabad,Cocktail Bar,Metro Station,Market,Fish Market,Electronics Store,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,6047.0,17.476746,78.422108,5
8,Banjara Hills Hyderabad,Coffee Shop,Furniture / Home Store,Indian Restaurant,Bakery,Sandwich Place,Shoe Store,Boutique,Ice Cream Shop,Deli / Bodega,Bar,29600.0,17.417746,78.439901,8
9,Banjara Hills Hyderabad,Coffee Shop,Furniture / Home Store,Indian Restaurant,Bakery,Sandwich Place,Shoe Store,Boutique,Ice Cream Shop,Deli / Bodega,Bar,29600.0,17.417746,78.439901,8


In [29]:
df_left.set_index("Neighborhood", inplace = True)
df_left.drop(['latitude','longitude'],axis=1,inplace=True)
df_left

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Avg Price per sqft,Distance in kms
Neighborhood,,,,,,,,,,,,
AP SE Board Hyderabad,Indian Restaurant,Fast Food Restaurant,Bakery,Shop & Service,Clothing Store,Bus Station,Pizza Place,Breakfast Spot,Sandwich Place,Middle Eastern Restaurant,4445.0,4
Ac Guards Hyderabad,Hyderabadi Restaurant,Hotel,Bakery,Women's Store,Deli / Bodega,Electronics Store,Dumpling Restaurant,Diner,Dessert Shop,Department Store,6492.0,11
Ahmed Nagar Hyderabad,Snack Place,Café,Indian Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Women's Store,Dumpling Restaurant,Diner,Dessert Shop,Department Store,2343.0,9
Amberpet Hyderabad,Movie Theater,IT Services,Indian Restaurant,Women's Store,Farmers Market,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Currency Exchange,2222.0,17
Ameerpet Hyderabad,Indian Restaurant,Supermarket,Hotel,Diner,Buffet,Food Court,Sandwich Place,Restaurant,Vegetarian / Vegan Restaurant,Cupcake Shop,9628.0,8
Ashok Nagar Hyderabad,Park,Restaurant,Dessert Shop,Brewery,Women's Store,Cocktail Bar,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,6106.0,13
BK Guda Hyderabad,Indian Restaurant,Bakery,Snack Place,Women's Store,Dumpling Restaurant,Diner,Dessert Shop,Department Store,Deli / Bodega,Farmers Market,3034.0,7
Balanagar Hyderabad,Cocktail Bar,Metro Station,Market,Fish Market,Electronics Store,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,6047.0,5
Banjara Hills Hyderabad,Coffee Shop,Furniture / Home Store,Indian Restaurant,Bakery,Sandwich Place,Shoe Store,Boutique,Ice Cream Shop,Deli / Bodega,Bar,29600.0,8


In [39]:
# set number of clusters
kclusters = 7

hyderabad_grouped_clustering = hyderabad_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hyderabad_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_



array([0, 4, 0, 0, 0, 2, 0, 2, 4, 4, 4, 4, 4, 0, 4, 2, 1, 6, 4, 0, 2, 0,
       4, 1, 0, 2, 4, 2, 0, 4, 2, 0, 4, 2, 0, 2, 4, 2, 4, 0, 0, 0, 0, 4,
       0, 4, 2, 4, 0, 4, 0, 0, 3, 0, 4, 0, 0, 5, 0, 0, 2, 4, 4, 4, 4, 2,
       3, 0, 4, 0, 0, 4, 4, 0, 4, 4, 3, 0, 2, 4], dtype=int32)

In [40]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels']=kmeans.labels_.tolist()

hyderabad_merged = hyderabad_neighbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hyderabad_merged = pd.merge(neighborhoods_venues_sorted,hyderabad_merged,on='Neighborhood',how = 'outer')

#neighborhoods_venues_sorted.insert(0, 'sfdafgdfg Labels', kmeans.labels_)
hyderabad_merged.dropna(how='any',inplace=True)# check the last columns!

In [41]:
hyderabad_merged["Cluster Labels"]=hyderabad_merged["Cluster Labels"].astype(int)

In [42]:
# create map
map_clusters = folium.Map(location=[office_latitude, office_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
 
    
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hyderabad_merged['latitude'], hyderabad_merged['longitude'], hyderabad_merged['Neighborhood'], hyderabad_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
#office pin
label = 'Google'
folium.Marker(
    [office_latitude, office_longitude],
    radius=10,
    popup=label,
    icon=folium.Icon(color='red')
    ).add_to(map_clusters) 
       
map_clusters